# Erosion Simulation - Notebook Version

This notebook runs the erosion simulation with realistic multi-layer geology and weather patterns.

## 1. Setup - Add Workspace to Path

In [ ]:
# Add workspace to Python path so we can import erosion_simulation
import sys
from pathlib import Path

workspace = Path("/workspace")
if str(workspace) not in sys.path:
    sys.path.insert(0, str(workspace))
    print(f"✓ Added {workspace} to Python path")
else:
    print(f"✓ Workspace already in path")

## 2. Import Modules

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import erosion simulation
from erosion_simulation import (
    ErosionSimulation,
    plot_simulation_summary,
    plot_topography,
    ERODIBILITY
)

print("✓ All imports successful")
print(f"✓ Loaded {len(ERODIBILITY)} material types with erodibility data")

## 3. Generate Terrain

In [ ]:
def generate_terrain(N=128, seed=42):
    """Generate realistic terrain using fractional Brownian motion."""
    np.random.seed(seed)
    
    # Power-law spectrum
    kx = np.fft.fftfreq(N)
    ky = np.fft.rfftfreq(N)
    K = np.sqrt(kx[:, None]**2 + ky[None, :]**2)
    K[0, 0] = np.inf
    
    beta = 3.0
    amp = 1.0 / (K ** (beta/2))
    phase = np.random.uniform(0, 2*np.pi, size=(N, ky.size))
    spec = amp * (np.cos(phase) + 1j*np.sin(phase))
    spec[0, 0] = 0.0
    
    z = np.fft.irfftn(spec, s=(N, N))
    z = (z - z.min()) / (z.max() - z.min())
    
    # Add mountain features
    xx, yy = np.meshgrid(np.linspace(-1, 1, N), np.linspace(-1, 1, N))
    mountains = 0.3 * np.exp(-2 * (xx**2 + yy**2))
    
    z = 0.7 * z + 0.3 * mountains
    return z * 1500.0  # Scale to meters

# Generate terrain
N = 128
surface_elevation = generate_terrain(N=N, seed=42)

print(f"✓ Generated {N}×{N} terrain")
print(f"  Elevation range: {surface_elevation.min():.1f} to {surface_elevation.max():.1f} m")

# Visualize
plt.figure(figsize=(10, 8))
plt.imshow(surface_elevation, origin='lower', cmap='terrain')
plt.colorbar(label='Elevation (m)')
plt.title('Generated Terrain')
plt.show()

## 4. Generate Geological Layers

In [ ]:
def generate_layers(surface_elevation):
    """Generate geological layer stack."""
    ny, nx = surface_elevation.shape
    
    # Compute slope factor
    dy, dx = np.gradient(surface_elevation)
    slope = np.sqrt(dx**2 + dy**2)
    slope_factor = np.clip(1.0 - slope / 0.3, 0.2, 1.0)
    
    # Simple layers
    layer_order = [
        "Topsoil",
        "Subsoil",
        "Sandstone",
        "Shale",
        "Limestone",
        "Granite",
        "Basement"
    ]
    
    current_elev = surface_elevation.copy()
    layer_interfaces = {}
    
    # Create layers
    thicknesses = [2, 3, 50, 100, 120, 300, 500]
    
    for i, (name, thick) in enumerate(zip(layer_order, thicknesses)):
        if i < 2:  # Surface layers affected by slope
            thick_array = thick * slope_factor
        else:
            thick_array = thick
        
        current_elev = current_elev - thick_array
        layer_interfaces[name] = current_elev.copy()
    
    return layer_interfaces, layer_order

# Generate layers
layer_interfaces, layer_order = generate_layers(surface_elevation)

print(f"✓ Generated {len(layer_interfaces)} geological layers")
print(f"  Layers (top to bottom): {', '.join(layer_order)}")

# Show erodibility
print("\n  Erodibility coefficients:")
for layer in layer_order:
    K = ERODIBILITY.get(layer, ERODIBILITY['Unknown'])
    print(f"    {layer:12s}: K = {K:.6f}")

## 5. Initialize Erosion Simulation

In [ ]:
# Initialize simulation
sim = ErosionSimulation(
    surface_elevation=surface_elevation,
    layer_interfaces=layer_interfaces,
    layer_order=layer_order,
    pixel_scale_m=100.0,
    uplift_rate=0.0001  # 0.1 mm/year
)

print("✓ Erosion simulation initialized")
print(f"  Grid: {sim.nx} × {sim.ny} cells")
print(f"  Resolution: {sim.pixel_scale_m} m/pixel")
print(f"  Domain size: {sim.nx * sim.pixel_scale_m / 1000:.1f} × {sim.ny * sim.pixel_scale_m / 1000:.1f} km")
print(f"  Uplift rate: {sim.uplift_rate * 1000:.3f} mm/year")

## 6. Run Erosion Simulation

In [ ]:
# Simulation parameters
n_years = 50
mean_annual_rain_mm = 1200.0

print(f"Running {n_years}-year simulation...")
print(f"Mean annual rainfall: {mean_annual_rain_mm} mm/year\n")

# Run simulation
for year in range(n_years):
    # Generate rainfall (simple orographic pattern)
    elev_norm = (sim.elevation - sim.elevation.min()) / \
                (sim.elevation.max() - sim.elevation.min() + 1e-9)
    rainfall_map = mean_annual_rain_mm * (0.5 + 0.5 * elev_norm)
    
    # Add random storm
    if np.random.random() < 0.2:
        storm_row = np.random.randint(0, sim.ny)
        storm_col = np.random.randint(0, sim.nx)
        rows = np.arange(sim.ny)
        cols = np.arange(sim.nx)
        R, C = np.meshgrid(rows, cols, indexing='ij')
        dist_sq = (R - storm_row)**2 + (C - storm_col)**2
        storm_pattern = 5.0 * np.exp(-dist_sq / (2 * 40**2))
        rainfall_map = rainfall_map * (1.0 + storm_pattern)
    
    # Apply erosion
    sim.step(dt=1.0, rainfall_map=rainfall_map)
    
    # Progress
    if (year + 1) % 10 == 0:
        print(f"  Year {year+1:3d}: Erosion = {sim.get_total_erosion()/1e6:.2f} km³, "
              f"Rivers = {np.sum(sim.river_mask):3d} cells, "
              f"Lakes = {np.sum(sim.lake_mask):2d} cells")

print("\n✓ Simulation complete!")

## 7. Results and Statistics

In [ ]:
print("Final Statistics:")
print(f"  Duration: {sim.current_time:.1f} years")
print(f"  Total erosion: {sim.get_total_erosion()/1e9:.4f} km³")
print(f"  Total deposition: {sim.get_total_deposition()/1e9:.4f} km³")
print(f"  River cells: {np.sum(sim.river_mask)} ({100*np.sum(sim.river_mask)/sim.river_mask.size:.2f}%)")
print(f"  Lake cells: {np.sum(sim.lake_mask)} ({100*np.sum(sim.lake_mask)/sim.lake_mask.size:.2f}%)")
print(f"  Mean elevation change: {(sim.elevation - surface_elevation).mean():.2f} m")
print(f"  Max erosion depth: {(surface_elevation - sim.elevation).max():.2f} m")

## 8. Visualization

In [ ]:
# Comprehensive summary plot
fig = plot_simulation_summary(sim)
plt.show()

## 9. Detailed Analysis

In [ ]:
# Create custom detailed plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Initial terrain
im1 = axes[0, 0].imshow(surface_elevation, origin='lower', cmap='terrain')
plt.colorbar(im1, ax=axes[0, 0], label='Elevation (m)')
axes[0, 0].set_title('Initial Terrain')

# Final terrain
im2 = axes[0, 1].imshow(sim.elevation, origin='lower', cmap='terrain')
plt.colorbar(im2, ax=axes[0, 1], label='Elevation (m)')
axes[0, 1].set_title(f'Final Terrain (after {n_years} years)')

# Elevation change
change = sim.elevation - surface_elevation
vmax = np.abs(change).max()
im3 = axes[1, 0].imshow(change, origin='lower', cmap='RdBu', vmin=-vmax, vmax=vmax)
plt.colorbar(im3, ax=axes[1, 0], label='Change (m)')
axes[1, 0].set_title('Elevation Change (Red=Deposition, Blue=Erosion)')

# Rivers and lakes
axes[1, 1].imshow(sim.elevation, origin='lower', cmap='gray', alpha=0.3)
if np.any(sim.river_mask):
    river_y, river_x = np.where(sim.river_mask)
    axes[1, 1].scatter(river_x, river_y, c='blue', s=5, alpha=0.6, label='Rivers')
if np.any(sim.lake_mask):
    lake_y, lake_x = np.where(sim.lake_mask)
    axes[1, 1].scatter(lake_x, lake_y, c='cyan', s=10, alpha=0.8, label='Lakes')
axes[1, 1].set_title('Water Features')
if np.any(sim.river_mask) or np.any(sim.lake_mask):
    axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 10. Surface Materials

In [ ]:
# Show what materials are exposed at the surface
surface_materials = sim.get_surface_material()
unique_materials = np.unique(surface_materials)

print("Surface materials exposed after erosion:")
for mat in unique_materials:
    count = np.sum(surface_materials == mat)
    percent = 100 * count / surface_materials.size
    K = ERODIBILITY.get(mat, ERODIBILITY['Unknown'])
    print(f"  {mat:12s}: {count:5d} cells ({percent:5.2f}%), K = {K:.6f}")

## Summary

This notebook demonstrates:
- ✓ Realistic terrain generation
- ✓ Multi-layer geology with different erodibility
- ✓ Rainfall-driven erosion
- ✓ River and lake formation
- ✓ Long-term landscape evolution

You can customize:
- Grid size (`N`)
- Simulation duration (`n_years`)
- Rainfall amount (`mean_annual_rain_mm`)
- Layer types and thicknesses
- Uplift rate